In [1]:
from cc.model import NonlinearModel, NonlinearModelOptions
from cc.model.eval_model import eval_model
from cc.collect import collect_sample
from cc.env import make_env, ModelBasedEnv
from cc.train import train_model, TrainingOptionsModel
from cc import save, load 
import jax.random as jrand
import optax  

/home/simon/miniforge3/envs/chain_control/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/simon/miniforge3/envs/chain_control/lib/python3.9/site-packages/beartype/_util/hint/pep/utilpeptest.py:345: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Callable deprecated by PEP 585 scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". See this discussion for further details and alternatives:
    https://github.com/beartype/beartype#pep-585-deprecations
  warn(


In [2]:
time_limit = 10.0
control_timestep = 0.01

env = make_env("two_segments_v2", time_limit=time_limit, control_timestep=control_timestep, random=1)

In [13]:
train_sample = collect_sample(
    env,
    seeds_gp=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14],
    seeds_cos=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
)

test_sample = collect_sample(
    env, 
    seeds_gp=[15, 16, 17, 18, 19],
    seeds_cos=[2.5, 5.0, 7.5, 1.0, 12.5] # really shouldn't be called seeds, rather frequency
)

In [60]:
import jax.numpy as jnp 

In [64]:
options = NonlinearModelOptions(
    15, 1, 1, # state-size, input-size, output-size
    "EE", # integrate-method
    jrand.PRNGKey(1,), # seed for parameter init
    depth_f=2, # number of layers 
    width_f=35, # width of layers
    depth_g=0,
    use_dropout_f=True,
    dropout_rate_f=0.2,
    reset_key=True,
    act_fn_f = jnp.arctan,
    act_final_f = jnp.arctan
)

In [65]:
model = NonlinearModel(options)

In [66]:
import numpy as np


action = np.array([0.2])
# this returns a new model with an updated internal state
# and of course the actual prediction of the observation
new_model, predicted_obs = model(action)

                If this message is display continuously then you probably forgot to compile the model or controller. 
                This can be fixed by calling `*model/controller* = equniox.filter_jit(*model/controller*).
                


In [67]:
predicted_obs

OrderedDict([('xpos_of_segment_end',
              DeviceArray([-0.2472117], dtype=float32))])

In [68]:
type(new_model)

cc.model.model.NonlinearModel

In [69]:
training_options = TrainingOptionsModel(
    jrand.PRNGKey(1,), optax.adam(3e-3), 0.05, 1500, 1, True 
)

# requires ~40 seconds on my PC
# achieves a Test-Loss of ~4.2 on v2
model, losses = train_model(model, train_sample, training_options=training_options, test_sample=test_sample)

  0%|          | 0/1500 [00:00<?, ?it/s]

                If this message is display continuously then you probably forgot to compile the model or controller. 
                This can be fixed by calling `*model/controller* = equniox.filter_jit(*model/controller*).
                
                If this message is display continuously then you probably forgot to compile the model or controller. 
                This can be fixed by calling `*model/controller* = equniox.filter_jit(*model/controller*).
                
                If this message is display continuously then you probably forgot to compile the model or controller. 
                This can be fixed by calling `*model/controller* = equniox.filter_jit(*model/controller*).
                
                If this message is display continuously then you probably forgot to compile the model or controller. 
                This can be fixed by calling `*model/controller* = equniox.filter_jit(*model/controller*).
                


Trainings-Loss:   228.0504 | Test-Loss:   697.7700 | ParamsRegu:     0.0124:   0%|          | 1/1500 [00:07<3:14:07,  7.77s/it]

                If this message is display continuously then you probably forgot to compile the model or controller. 
                This can be fixed by calling `*model/controller* = equniox.filter_jit(*model/controller*).
                
                If this message is display continuously then you probably forgot to compile the model or controller. 
                This can be fixed by calling `*model/controller* = equniox.filter_jit(*model/controller*).
                
                If this message is display continuously then you probably forgot to compile the model or controller. 
                This can be fixed by calling `*model/controller* = equniox.filter_jit(*model/controller*).
                
                If this message is display continuously then you probably forgot to compile the model or controller. 
                This can be fixed by calling `*model/controller* = equniox.filter_jit(*model/controller*).
                


Trainings-Loss:     0.6096 | Test-Loss:    28.0406 | ParamsRegu:     0.0328: 100%|██████████| 1500/1500 [04:27<00:00,  5.62it/s]


In [56]:
model = model.reset()

In [57]:
# small little helper function
predicted_observation, test_rmse = eval_model(model, test_sample)

                If this message is display continuously then you probably forgot to compile the model or controller. 
                This can be fixed by calling `*model/controller* = equniox.filter_jit(*model/controller*).
                


In [58]:
predicted_observation["xpos_of_segment_end"].shape

(10, 1001, 1)

In [59]:
test_rmse

OrderedDict([('xpos_of_segment_end', DeviceArray(2.787502, dtype=float32))])

Finally, you can also replace the `Mujoco` physics component in your environment with your model. 

This creates a new environment that looks exactly the same from outside.

In [53]:
env_model = ModelBasedEnv(env, model, time_limit=time_limit, control_timestep=control_timestep)

In [54]:
env_model.step([0.2])

TimeStep(step_type=<StepType.MID: 1>, reward=array(0.), discount=array(1., dtype=float32), observation=OrderedDict([('xpos_of_segment_end', array([-0.07805266], dtype=float32))]))

In [55]:
model

NonlinearModel(
  rhs=NonlinearRHS(
    f=Sequential(
      layers=(
        Dropout(p=0.2, inference=False),
        Linear(
          weight=f32[25,13],
          bias=f32[25],
          in_features=13,
          out_features=25,
          use_bias=True
        ),
        Lambda(fn=<wrapped function relu>),
        Dropout(p=0.2, inference=False),
        Linear(
          weight=f32[25,25],
          bias=f32[25],
          in_features=25,
          out_features=25,
          use_bias=True
        ),
        Lambda(fn=<wrapped function relu>),
        Dropout(p=0.2, inference=False),
        Linear(
          weight=f32[12,25],
          bias=f32[12],
          in_features=25,
          out_features=12,
          use_bias=True
        ),
        Dropout(p=0.2, inference=False),
        Lambda(fn=<function <lambda>>)
      )
    ),
    g=Sequential(
      layers=(
        Linear(
          weight=f32[1,12],
          bias=f32[1],
          in_features=12,
          out_features=1,
  